# Minimal SPDIM incorporating TSMNet demo notebook for inter-session/-subject source-free unsupervised domain adaptation (SFUDA) under label shifts

In [1]:
import torch
import sklearn
import pandas as pd
from copy import deepcopy
from moabb.datasets import BNCI2015_001
from moabb.paradigms import MotorImagery
from spdnets.dataloader import StratifiedDomainDataLoader, DomainDataset 
from spdnets.models import TSMNet
import spdnets.batchnorm as bn
import spdnets.functionals as fn
from spdnets.trainer import Trainer
from spdnets.callbacks import MomentumBatchNormScheduler, EarlyStopping

## Parameters for experiments
### Notice: define the evaluation setting (i.e., inter-session/inter-subject) and the label ratio (label shifts level in the target domain) here.
### We have provided pre-trained source models. If you wish to train the model from scratch, please set 'pretrained_model' to False. It usually takes 5/30 mins for inter-session/inter-subject to train on standard PCs with a single GPU.

In [2]:
# Network and training configuration
cfg = dict(
    # parameters for experiments
    epochs = 100,
    batch_size_train = 50,
    domains_per_batch = 5,
    validation_size = 0.2,
    evaluation = 'inter-session', # 'inter-subject' or 'inter-session'
    label_ratio = 0.2,        # we set 0.2 in the paper
    dtype = torch.float32,
    pretrained_model = True,
    # parameters for the TSMNet model
    mdl_kwargs = dict(
        temporal_filters=4,
        spatial_filters=40,
        subspacedims=20, 
        bnorm_dispersion=bn.BatchNormDispersion.SCALAR,
        spd_device='cpu',
        spd_dtype=torch.double,
        domain_adaptation=True
    )
)

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU')
else:
    device = torch.device('cpu')
    print('CPU')

GPU


## load a MOABB dataset. 
### Notice: there is no need to manually download and preprocess the datasets. This is done automatically in MOABB pipeline

In [3]:
moabb_ds = BNCI2015_001()
n_classes = 2
moabb_paradigm = MotorImagery(n_classes=n_classes, events=['right_hand', 'feet'], fmin=4, fmax=36, tmin=1.0, tmax=4.0, resample=256)

## fit and evaluat the model for all domains

In [4]:
records = []

# Check the evaluation type in the configuration
if 'inter-session' in cfg['evaluation']:
    subset_iter = iter([[s] for s in moabb_ds.subject_list])
    groupvarname = 'session'
elif 'inter-subject' in cfg['evaluation']:
    subset_iter = iter([None])
    groupvarname = 'subject'
else:
    raise NotImplementedError()


# iterate over groups
for ix_subset, subjects in enumerate(subset_iter):

    # get the data from the MOABB paradigm/dataset
    X, labels, metadata = moabb_paradigm.get_data(moabb_ds, subjects=subjects, return_epochs=False)

    # extract domains = subject/session
    metadata['label'] = labels
    metadata['domain'] = metadata.apply(lambda row: f'{row.subject}/{row.session}',  axis=1)
    domain = sklearn.preprocessing.LabelEncoder().fit_transform(metadata['domain'])

    # convert to torch tensors
    domain = torch.from_numpy(domain)
    X = torch.from_numpy(X)
    y = sklearn.preprocessing.LabelEncoder().fit_transform(labels)
    y = torch.from_numpy(y)

    # leave one subject or session out
    cv_outer = sklearn.model_selection.LeaveOneGroupOut()
    cv_outer_group = metadata[groupvarname]

    # train/validation split stratified across domains and labels
    cv_inner_group = metadata.apply(lambda row: f'{row.domain}/{row.label}',  axis=1)
    cv_inner_group = sklearn.preprocessing.LabelEncoder().fit_transform(cv_inner_group)

    # add dataset depended model kwargs
    mdl_kwargs = deepcopy(cfg['mdl_kwargs'])
    mdl_kwargs['nclasses'] = n_classes
    mdl_kwargs['nchannels'] = X.shape[1]
    mdl_kwargs['nsamples'] = X.shape[2]
    mdl_kwargs['domains'] = domain.unique()

    # perform outer CV
    for ix_fold, (fit, test) in enumerate(cv_outer.split(X, y, cv_outer_group)):

        # split fitting data into train and validation 
        cv_inner = sklearn.model_selection.StratifiedShuffleSplit(n_splits=1, test_size=cfg['validation_size'])
        train, val = next(cv_inner.split(X[fit], y[fit], cv_inner_group[fit]))

        # adjust number of domains if necessary
        du = domain[fit][train].unique()
        if cfg['domains_per_batch'] > len(du):
            domains_per_batch = len(du)
        else:
            domains_per_batch = cfg['domains_per_batch']

        # get the label ratio , here source domain is balanced
        source_label_ratio, target_label_ratio = fn.get_label_ratio(y, cfg['label_ratio'])
        
        # split entire dataset into train/validation
        ds_train = DomainDataset(X[fit][train], y[fit][train], domain[fit][train],label_ratio=source_label_ratio)
        ds_val = DomainDataset(X[fit][val], y[fit][val], domain[fit][val], label_ratio=source_label_ratio) 

        # create dataloaders, for training use specific loader/sampler so that 
        # batches contain a specific number of domains with equal observations per domain and stratified labels       
        loader_train = StratifiedDomainDataLoader(ds_train, cfg['batch_size_train'], domains_per_batch=domains_per_batch, shuffle=True)
        loader_val = torch.utils.data.DataLoader(ds_val, batch_size=len(ds_val))

        # create the model
        net = TSMNet(**mdl_kwargs).to(device=device, dtype=cfg['dtype'])

        # create the momentum scheduler and early stopping callback
        bn_sched = MomentumBatchNormScheduler(
            epochs=cfg['epochs']-10,
            bs0=cfg['batch_size_train'],
            bs=cfg['batch_size_train']/cfg['domains_per_batch'], 
            tau0=0.85
        )
        es = EarlyStopping(metric='val_loss', higher_is_better=False, patience=20, verbose=False)
        
        # create the trainer
        trainer = Trainer(
            max_epochs=cfg['epochs'],
            min_epochs=50,
            callbacks=[bn_sched, es],
            loss= torch.nn.CrossEntropyLoss(weight = None),
            device=device, 
            dtype=cfg['dtype']
        )

        # fit the model extract model parameters
        parameter_t = torch.tensor(1,dtype=torch.float64,device='cpu')

        if cfg['pretrained_model']:
            if cfg['evaluation'] == 'inter-session':
                state_dict = torch.load(f"pretrained_model/session/state_dict_{ix_subset}{ix_fold}.pt", map_location=device)
            elif cfg['evaluation'] == 'inter-subject':
                state_dict = torch.load(f"pretrained_model/subject/state_dict_{ix_fold}.pt", map_location=device)
        else:
            trainer.fit(net, train_dataloader=loader_train, val_dataloader=loader_val,parameter_t=parameter_t)
            state_dict = deepcopy(net.state_dict())

        # create a new model for SFUDA
        sfuda_offline_net = TSMNet(**mdl_kwargs).to(device=device)
        sfuda_offline_net.load_state_dict(state_dict)
        test_domain=domain[test].unique()


        # Evaluate over test domains in the target domain 
        for test_domain in test_domain:
            if 'inter-session' in cfg['evaluation']:
                subject=ix_subset
            else:
                subject=ix_fold
            print(f"Subject:{subject}, test domain: {test_domain}")
            
            # create test dataset, and artificially introduce the label shifts
            ds_test = DomainDataset(X[test][domain[test] == test_domain], y[test][domain[test] == test_domain], domain[test][domain[test] == test_domain], label_ratio=target_label_ratio)
            loader_test = torch.utils.data.DataLoader(ds_test, batch_size=len(ds_test))


            # enable SFUDA 
            sfuda_offline_net.eval()
            sfuda_offline_net.domainadapt_finetune(ds_test.features.to(dtype=cfg['dtype'], device=device), ds_test.labels.to(device=device), ds_test.domains, 'refit')

            # SFUDA method: RCT
            res = trainer.test(sfuda_offline_net, dataloader=loader_test,parameter_t=parameter_t)
            print('RCT',res)
            records.append(dict(mode='RCT',subject=subject,domain=test_domain, **res))

            # SFUDA method: clustering refined mean [Li et al. 2024, ESANN]
            refined_mean = trainer.get_refined_mean(sfuda_offline_net, test_dataloader=loader_test,parameter_t=parameter_t)
            res = trainer.test(sfuda_offline_net, dataloader=loader_test, parameter_t=parameter_t,fm_mean=refined_mean)
            print('clustering',res)
            records.append(dict(mode="clustering",subject=subject,domain=test_domain, **res))           




C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 200 events (all good), 1 – 4 s (baseline off), ~30.5 MB, data loaded,
 'right_hand': 100
 'feet': 100>
  warn(f"warnEpochs {epochs}")
C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 200 events (all good), 1 – 4 s (baseline off), ~30.5 MB, data loaded,
 'right_hand': 100
 'feet': 100>
  warn(f"warnEpochs {epochs}")
C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:2335: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(


Subject:0, test domain: 0
RCT {'loss': 0.1862202286720276, 'score': 0.96}
clustering {'loss': 0.00427457457408309, 'score': 1.0}
Subject:0, test domain: 1
RCT {'loss': 0.2871244549751282, 'score': 0.9450000000000001}


C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:2335: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(


clustering {'loss': 0.044719304889440536, 'score': 0.99}


C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 200 events (all good), 1 – 4 s (baseline off), ~30.5 MB, data loaded,
 'right_hand': 100
 'feet': 100>
  warn(f"warnEpochs {epochs}")
C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 200 events (all good), 1 – 4 s (baseline off), ~30.5 MB, data loaded,
 'right_hand': 100
 'feet': 100>
  warn(f"warnEpochs {epochs}")
C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:2335: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(


...

clustering {'loss': 0.7330495119094849, 'score': 0.86}


C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 200 events (all good), 1 – 4 s (baseline off), ~30.5 MB, data loaded,
 'right_hand': 100
 'feet': 100>
  warn(f"warnEpochs {epochs}")
C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 200 events (all good), 1 – 4 s (baseline off), ~30.5 MB, data loaded,
 'right_hand': 100
 'feet': 100>
  warn(f"warnEpochs {epochs}")
C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:2335: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(


Subject:11, test domain: 0
RCT {'loss': 0.9370726943016052, 'score': 0.735}
clustering {'loss': 1.1962510347366333, 'score': 0.69}
Subject:11, test domain: 1
RCT {'loss': 0.6992105841636658, 'score': 0.655}


C:\Users\shanglin\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:2335: UserWarning: The groups parameter is ignored by StratifiedShuffleSplit
  warnings.warn(


clustering {'loss': 0.6393830180168152, 'score': 0.685}


In [5]:
resdf = pd.DataFrame(records)
resdf.groupby(['mode']).agg(['mean', 'std']).round(4)

subject          domain            loss           score        
              mean     std    mean     std    mean     std    mean     std
mode                                                                      
RCT         5.9286  3.4526  0.7143  0.7127  0.6245  0.2174  0.8041  0.0891
clustering  5.9286  3.4526  0.7143  0.7127  0.4885  0.2986  0.8238  0.1042